<a href="https://colab.research.google.com/github/Bobbsicle27/VandanaK/blob/main/CHAIR_STATS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clinician monthly chair occupancy

In [ ]:
import pandas as pd
from google.colab import files

# Helper function to upload files with error handling
def upload_file(prompt_message):
    print(prompt_message)
    uploaded_file = files.upload()
    if len(uploaded_file) == 0:
        raise ValueError("No file uploaded. Please upload the correct file.")
    return list(uploaded_file.keys())[0]

# Step 1: Upload the appointment report file
appointment_file_name = upload_file("Please upload the Appointment Report file:")
clinician_file_name = upload_file("Please upload the Clinician Working Hours Report file:")

# Load the appointment data
appointment_excel = pd.ExcelFile(appointment_file_name)
clinician_excel = pd.ExcelFile(clinician_file_name)

# Print sheet names for user reference
print("Appointment Report Sheet Names:", appointment_excel.sheet_names)
print("Clinician Report Sheet Names:", clinician_excel.sheet_names)

# Load the data from the first sheet (assuming headers are in the second row)
appointment_data = pd.read_excel(appointment_file_name, sheet_name=0, header=1)
clinician_data = pd.read_excel(clinician_file_name, sheet_name=0, header=1)

# Clean and prepare the appointment data
appointment_data['Duration'] = pd.to_numeric(appointment_data['Duration'], errors='coerce')
appointment_data['Clinician'] = appointment_data['Clinician'].str.strip()
appointment_data['Date'] = pd.to_datetime(appointment_data['Date'], format='%d-%m-%Y', errors='coerce')

# Filter completed appointments and extract Month/Year
completed_appointments = appointment_data[appointment_data['Appointment Status'] == 'Completed']
completed_appointments['Month'] = completed_appointments['Date'].dt.month
completed_appointments['Year'] = completed_appointments['Date'].dt.year

# Clean and prepare the clinician working hours data
clinician_data.columns = ['Date', 'Clinician', 'Total Scheduled Hours', 'Time Off', 'Total Appointment Hours',
                          'Events', 'Total Available Hours']
clinician_data['Clinician'] = clinician_data['Clinician'].str.strip()
clinician_data['Date'] = pd.to_datetime(clinician_data['Date'], format='%d-%m-%Y', errors='coerce')
clinician_data['Month'] = clinician_data['Date'].dt.month
clinician_data['Year'] = clinician_data['Date'].dt.year

# Convert times to timedelta and calculate working hours in minutes
clinician_data['Total Scheduled Hours'] = pd.to_timedelta(clinician_data['Total Scheduled Hours'] + ':00', errors='coerce')
clinician_data['Events'] = pd.to_timedelta(clinician_data['Events'] + ':00', errors='coerce')
clinician_data['Actual Working Hours (Minutes)'] = (
    (clinician_data['Total Scheduled Hours'] + clinician_data['Events']).dt.total_seconds() / 60
)

# Function to calculate chair occupancy
def calculate_monthly_chair_occupancy(completed_appointments, clinician_data):
    total_duration_per_clinician_month = (
        completed_appointments.groupby(['Clinician', 'Year', 'Month'])['Duration']
        .sum()
        .reset_index()
        .rename(columns={'Duration': 'Total Appointment Duration (Minutes)'})
    )
    total_working_hours_per_clinician_month = (
        clinician_data.groupby(['Clinician', 'Year', 'Month'])['Actual Working Hours (Minutes)']
        .sum()
        .reset_index()
    )
    utilization_df = pd.merge(total_duration_per_clinician_month, total_working_hours_per_clinician_month,
                              on=['Clinician', 'Year', 'Month'], how='inner')
    utilization_df['Percentage Chair Utilization'] = (
        (utilization_df['Total Appointment Duration (Minutes)'] /
         utilization_df['Actual Working Hours (Minutes)']) * 100
    )
    return utilization_df

# Calculate the utilization
utilization_df_monthly = calculate_monthly_chair_occupancy(completed_appointments, clinician_data)

# Save the resulting dataframe into an Excel file
output_file_path = "clinician_chair_utilization_by_month_BB_03_25.xlsx"
utilization_df_monthly.to_excel(output_file_path, index=False)

# Download the file
files.download(output_file_path)


Please upload the Appointment Report file:


Saving AppointmentReport_BB_2025-04-02_11-21.xlsx to AppointmentReport_BB_2025-04-02_11-21.xlsx
Please upload the Clinician Working Hours Report file:


Saving ClinicianActual_BB_2025-04-02_11-23.xlsx to ClinicianActual_BB_2025-04-02_11-23 (1).xlsx
Appointment Report Sheet Names: ['Sheet 1 - AppointmentReport_202']
Clinician Report Sheet Names: ['Sheet 1 - ClinicianActual_2025-']


<ipython-input-5-a3696e811b7c>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  completed_appointments['Month'] = completed_appointments['Date'].dt.month
<ipython-input-5-a3696e811b7c>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  completed_appointments['Year'] = completed_appointments['Date'].dt.year


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>